# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846796842826                   -0.70    4.8         
  2   -7.852317129624       -2.26       -1.53    1.0   25.1ms
  3   -7.852612926460       -3.53       -2.55    1.5   27.6ms
  4   -7.852645937327       -4.48       -2.87    2.8   35.8ms
  5   -7.852646480912       -6.26       -3.15    1.0   26.3ms
  6   -7.852646679283       -6.70       -4.15    1.0   26.9ms
  7   -7.852646686676       -8.13       -4.96    2.0   33.5ms
  8   -7.852646686722      -10.33       -5.43    1.5   29.2ms
  9   -7.852646686726      -11.39       -5.49    1.2   28.9ms
 10   -7.852646686730      -11.48       -6.73    1.0   26.8ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846832674390                   -0.70           4.8         
  2   -7.852551763855       -2.24       -1.64   0.80    2.2    294ms
  3   -7.852638817502       -4.06       -2.74   0.80    1.0   24.2ms
  4   -7.852646450910       -5.12       -3.35   0.80    2.0   31.3ms
  5   -7.852646679199       -6.64       -4.20   0.80    2.0   30.6ms
  6   -7.852646686484       -8.14       -4.78   0.80    2.0   30.9ms
  7   -7.852646686724       -9.62       -5.71   0.80    1.5   28.9ms
  8   -7.852646686730      -11.24       -6.57   0.80    2.0   30.4ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.381895e+01     3.725281e+00
 * time: 0.06307482719421387
     1     1.489329e+00     2.003395e+00
 * time: 0.29857492446899414
     2    -1.311481e+00     2.391501e+00
 * time: 0.32401394844055176
     3    -3.475219e+00     2.172915e+00
 * time: 0.360015869140625
     4    -4.857740e+00     2.055730e+00
 * time: 0.39627885818481445
     5    -6.645557e+00     1.323757e+00
 * time: 0.43230700492858887
     6    -7.422399e+00     3.332871e-01
 * time: 0.46988701820373535
     7    -7.650557e+00     1.348821e-01
 * time: 0.49558186531066895
     8    -7.745350e+00     1.668567e-01
 * time: 0.5211679935455322
     9    -7.795677e+00     9.572239e-02
 * time: 0.5470869541168213
    10    -7.823606e+00     8.093030e-02
 * time: 0.5728938579559326
    11    -7.836834e+00     6.670729e-02
 * time: 0.5989439487457275
    12    -7.844327e+00     5.982820e-02
 * time: 0.6244180202484131
    13    -7.846892e+00     5.412367e-02
 * time: 0.6503

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846849339414                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645910445                   -1.64         
  2   -7.852646686730       -6.11       -3.71    2.14s
  3   -7.852646686730      -13.27       -7.25    192ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.956530376659243e-7
|ρ_newton - ρ_scfv| = 2.674758568790832e-7
|ρ_newton - ρ_dm|   = 5.676694411200215e-10
